In [ ]:
import numpy as np
import random
import tensorflow as tf
from scipy.stats import entropy
import scipy
import os

# Learning From Crowds

In [ ]:
class LearningFromCrowds():
    def __init__(self, classusers, items, itemclasslabels, featureusers, feature_num, itemfeaturelabels, itemcontents, itemtruelabels, nofc, datasplitidx, train_epoch, savemodelfilename, latentfea_dim):
        self.classusers = classusers
        self.featureusers = featureusers
        self.items = items
        self.classuser_num = len(classusers)
        self.featureuser_num = len(featureusers)
        self.item_num = len(items)
        self.content_dim = len(itemcontents[0])
        self.feature_num = feature_num
        self.latentfea_dim = latentfea_dim
        self.datasplitidx = datasplitidx
        self.selfdata = convertlabels(classusers, items, itemclasslabels, featureusers, itemfeaturelabels, self.feature_num, itemcontents, itemtruelabels, nofc, self.datasplitidx)
        self.nofc = nofc
        self.itemtruelabels = itemtruelabels
        self.batch_size = 100
        self.train_epoch = train_epoch
        self.savemodelfilename = savemodelfilename

## LFCAD

In [ ]:
class LearningFromCrowds(CrowdNNLabelAggregation):
    def LFCAD_Model(self, lamb, lambre):
      content = tf.compat.v1.placeholder(tf.float64, shape=(None, self.content_dim), name='content')
      usersclasslabelmask = tf.compat.v1.placeholder(tf.float64, shape=(None, self.classuser_num, self.nofc), name='usersclasslabelmask')
      usersclasslabel = tf.compat.v1.placeholder(tf.float64, shape=(None, self.classuser_num, self.nofc), name='usersclasslabel')
      usersfeaturelabelmask = tf.compat.v1.placeholder(tf.float64, shape=(None, self.feature_num, self.featureuser_num), name='usersfeaturelabelmask')
      usersfeaturelabel = tf.compat.v1.placeholder(tf.float64, shape=(None, self.feature_num, self.featureuser_num), name='usersfeaturelabel')
    
      itemmvclasslabel = tf.compat.v1.placeholder(tf.float64, shape=(None, self.nofc), name='itemmvclasslabel')
      itemmvfeaturelabel = tf.compat.v1.placeholder(tf.float64, shape=(None, self.feature_num), name='itemmvfeaturelabel')
      
      with tf.compat.v1.variable_scope('lfcadmodel'):
          c2ilfwmean = tf.Variable(np.random.normal(size=(self.content_dim, self.latentfea_dim)), name='c2iflwmean')
          c2ilfbmean = tf.Variable(np.random.normal(size=(self.latentfea_dim)), name='c2iflbmean')
          c2ilfw2mean = tf.Variable(np.random.normal(size=(self.content_dim, self.latentfea_dim)), name='c2iflw2mean')
          c2ilfb2mean = tf.Variable(np.random.normal(size=(self.latentfea_dim)), name='c2iflb2mean')
          c2ilfwsharemean = tf.Variable(np.random.normal(size=(self.content_dim, self.latentfea_dim)), name='c2iflwsharemean')
          c2ilfbsharemean = tf.Variable(np.random.normal(size=(self.latentfea_dim)), name='c2iflbsharemean')
          c2ilfwothermean = tf.Variable(np.random.normal(size=(self.content_dim, self.latentfea_dim)), name='c2ilfwothermean')
          c2ilfbothermean = tf.Variable(np.random.normal(size=(self.latentfea_dim)), name='c2ilfbothermean')

          decoderw = tf.Variable(np.random.normal(size=(self.latentfea_dim*4, self.content_dim)), name='decoderw')
          decoderb = tf.Variable(np.random.normal(size=(self.content_dim)), name='decoderb')

          ilf2iffw = tf.Variable(np.random.normal(size=(self.latentfea_dim+self.latentfea_dim, self.feature_num)), name='ifl2iclw')
          ilf2iffb = tf.Variable(np.random.normal(size=(self.feature_num)), name='ifl2iclb')
          ilf2iclw2 = tf.Variable(np.random.normal(size=(self.latentfea_dim+self.latentfea_dim, self.nofc)), name='ifl2iclw2')
          ilf2iclb2 = tf.Variable(np.random.normal(size=(self.nofc)), name='ifl2iclb2')
            
          icl2ucl = tf.Variable(np.ones((self.nofc,self.nofc,self.classuser_num)), name='icl2ucl', constraint=lambda x: tf.clip_by_value(x, 0, np.inf))
          
          ifl2ufl = tf.Variable(np.ones((self.feature_num,self.feature_num,self.featureuser_num)), name='ifl2ufl', constraint=lambda x: tf.clip_by_value(x, 0, np.inf))
            

      itemlatentfeature1 = tf.tensordot(content, c2ilfwmean, axes = 1) + c2ilfbmean
      itemlatentfeature2 = tf.tensordot(content, c2ilfw2mean, axes = 1) + c2ilfb2mean
      itemsharefeature = tf.tensordot(content, c2ilfwsharemean, axes = 1) + c2ilfbsharemean
      itemotherfeature = tf.tensordot(content, c2ilfwothermean, axes = 1) + c2ilfbothermean

      itemfeaturelabelpred = tf.tensordot(tf.concat([itemlatentfeature1, itemsharefeature], 1), ilf2iffw, axes = 1) + ilf2iffb
      itemclasslabelpred = tf.tensordot(tf.concat([itemlatentfeature2, itemsharefeature], 1), ilf2iclw2, axes = 1) + ilf2iclb2
          
      # reconstruction loss
      reconstructedfeature = tf.tensordot(tf.concat([itemlatentfeature1, itemsharefeature, itemlatentfeature2, itemotherfeature], 1), decoderw, axes = 1) + decoderb
      reconstructionloss = tf.reduce_mean(tf.math.sqrt(tf.reduce_mean(tf.math.square(content-reconstructedfeature), 1)))
      
      # class label
      sofitemclasslabelpred = tf.math.softmax(itemclasslabelpred,1)
      usersclasslabelpred1 = tf.tensordot(itemclasslabelpred, icl2ucl, axes = 1)
      usersclasslabelpred = tf.transpose(usersclasslabelpred1, [0,2,1])
         
      sofusersclasslabelpred = tf.math.softmax(usersclasslabelpred,2)
      posusersclasslabelpred1 = -usersclasslabel*tf.math.log(sofusersclasslabelpred+1e-8)
      negusersclasslabelpred1 = -(1-usersclasslabel)*tf.math.log(1-sofusersclasslabelpred+1e-8)
      posnegusersclasspred1 = tf.expand_dims(tf.reduce_sum(usersclasslabelmask*(posusersclasslabelpred1+negusersclasslabelpred1),2), 2)
      usersclasslabelloss1 = tf.reduce_mean(posnegusersclasspred1)    
      posusersclasslabelpred2 = -usersclasslabel*tf.math.log(sofusersclasslabelpred+1e-8)
      negusersclasslabelpred2 = -(1-usersclasslabel)*tf.math.log(1-sofusersclasslabelpred+1e-8)
      posnegusersclasspred2 = tf.expand_dims(tf.reduce_sum(usersclasslabelmask*(posusersclasslabelpred2+negusersclasslabelpred2),2), 2)
      usersclasslabelloss2 = tf.reduce_mean(posnegusersclasspred2)
      usersclasslabelloss = usersclasslabelloss1 + usersclasslabelloss2
        
      # description label
      sigitemfeaturelabelpred = tf.math.sigmoid(itemfeaturelabelpred)
      usersfeaturelabelpred = tf.tensordot(itemfeaturelabelpred, ifl2ufl, axes = 1)
    
      sigusersfeaturelabelpred = tf.math.sigmoid(usersfeaturelabelpred)
      posusersfeaturelabelpred1 = -usersfeaturelabel*tf.math.log(sigusersfeaturelabelpred+1e-8)
      negusersfeaturelabelpred1 = -(1-usersfeaturelabel)*tf.math.log(1-sigusersfeaturelabelpred+1e-8)
      posnegusersfeaturepred1 = tf.expand_dims(tf.reduce_sum(usersfeaturelabelmask*(posusersfeaturelabelpred1+negusersfeaturelabelpred1),2), 2)
      usersfeaturelabelloss1 = tf.reduce_mean(posnegusersfeaturepred1)
      posusersfeaturelabelpred2 = -usersfeaturelabel*tf.math.log(sigusersfeaturelabelpred+1e-8)
      negusersfeaturelabelpred2 = -(1-usersfeaturelabel)*tf.math.log(1-sigusersfeaturelabelpred+1e-8)
      posnegusersfeaturepred2 = tf.expand_dims(tf.reduce_sum(usersfeaturelabelmask*(posusersfeaturelabelpred2+negusersfeaturelabelpred2),2), 2)
      usersfeaturelabelloss2 = tf.reduce_mean(posnegusersfeaturepred2)
      usersfeaturelabelloss = usersfeaturelabelloss1 + usersfeaturelabelloss2
    

      sofitemmvclasslabel = tf.math.softmax(itemmvclasslabel,1)
      positemclasslabelpred = -sofitemmvclasslabel*tf.math.log(sofitemclasslabelpred+1e-8)
      negitemclasslabelpred = -(1-sofitemmvclasslabel)*tf.math.log(1-sofitemclasslabelpred+1e-8)
      posnegitemclasslabelpred = tf.expand_dims(tf.reduce_sum(positemclasslabelpred+negitemclasslabelpred,1), 1)
      itemclasslabelloss = tf.reduce_mean(posnegitemclasslabelpred)
    
      sigitemmvfeaturelabel = tf.math.sigmoid(itemmvfeaturelabel)
      positemfeaturelabelpred = -sigitemmvfeaturelabel*tf.math.log(sigitemfeaturelabelpred+1e-8)
      negitemfeaturelabelpred = -(1-sigitemmvfeaturelabel)*tf.math.log(1-sigitemfeaturelabelpred+1e-8)
      posnegitemfeaturelabelpred = tf.expand_dims(tf.reduce_sum(positemfeaturelabelpred+negitemfeaturelabelpred,1), 1)
      itemfeaturelabelloss = tf.reduce_mean(posnegitemfeaturelabelpred)
        
      loss = 1 * usersclasslabelloss + lamb * usersfeaturelabelloss + lambre * reconstructionloss 
            
      return (content, usersclasslabel, usersclasslabelmask, usersfeaturelabel, usersfeaturelabelmask, itemclasslabelpred, itemfeaturelabelpred, itemsharefeature, sofusersclasslabelpred, usersclasslabelloss, sigusersfeaturelabelpred, usersfeaturelabelloss, loss, icl2ucl, ifl2ufl, itemmvclasslabel, itemclasslabelloss, itemmvfeaturelabel, itemfeaturelabelloss, reconstructionloss)